In [1]:
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['Company','Rating','Salary','Job','Speciality','Skills'])

# Чтобы спарсить все вакансии пока достаточно обойти 56 страниц
for i in range(56):
    html_page = urlopen("https://career.habr.com/vacancies?page="+str(i)+"&type=all&with_salary=true")
    soup = BeautifulSoup(html_page, "html.parser")
    for link in soup.findAll('div', class_='vacancy-card__inner'):
        df.loc[len(df)] = [
            link.find('div', class_='vacancy-card__company-title').text if link.find('div', class_='vacancy-card__company-title') else None,
            link.find('div', class_='vacancy-card__company-rating').text if link.find('div', class_='vacancy-card__company-rating') else None,
            link.find('div', class_='basic-salary').text if link.find('div', class_='basic-salary') else None,
            link.find('div', class_='vacancy-card__title').text if link.find('div', class_='vacancy-card__title') else None,
            link.find('div', class_='vacancy-card__skills').text.split('•')[0] if link.find('div', class_='vacancy-card__skills') else None,
            link.find('div', class_='vacancy-card__skills').text.split('•')[1:]] if link.find('div', class_='vacancy-card__skills') else None
backup_df = df.copy()

In [3]:
df['Skills'] = [[item.strip() for item in row] for row in df['Skills']]

df['MinSalary'] = \
df['Salary'].str.replace(' ','').str.extract(r'^(от)*([0-9]+)')[1] \
if df['Salary'].str.replace(' ','').str.extract(r'^(от)*([0-9]+)')[1] is not None \
else df['Salary'].str.replace(' ','').str.extract(r'([0-9]+)')[1]

df['MaxSalary'] = \
df['Salary'].str.replace(' ','').str.extract(r'(до)+([0-9]+)')[1] \
if df['Salary'].str.replace(' ','').str.extract(r'(до)+([0-9]+)')[1] is not None \
else df['Salary'].str.replace(' ','').str.extract(r'([0-9]+)')[1]

df['MinSalary'] = df['MinSalary'].mask(pd.isnull, df['MaxSalary'])
df['MaxSalary'] = df['MaxSalary'].mask(pd.isnull, df['MinSalary'])
df['AvgSalary'] = (df['MaxSalary'].astype('float')+df['MinSalary'].astype('float'))/2.
df['Currency'] = df['Salary'].str[-1]
# Необходима единственная валюта для расчета зарплат. Возьмем 1$=100₽ и 1€ = 100₽
df['RubAvgSalary'] = np.where(df['Currency'] == '₽', df['AvgSalary'], df['AvgSalary']*100)
df['SkillsCount'] = df['Skills'].apply(len)
df['SkillSalary'] = df['AvgSalary'] / df['SkillsCount']
df['Level'] = df['Speciality'].str.extract('(Intern|Junior|Middle|Senior|Lead)')

In [4]:
# Полученные данные выглядят следующим образом
df

,Company,Rating,Salary,Job,Speciality,Skills,MinSalary,MaxSalary,AvgSalary,Currency,RubAvgSalary,SkillsCount,SkillSalary,Level
0,Интер РАО – Управление сервисами,None,от 180 000 ₽,Ведущий инженер Linux,"Администратор серверов, Средний (Middle)","[Linux, Ansible, Puppet]",180000,180000,180000.0,₽,180000.0,3,60000.000000,Middle
1,Интер РАО – Управление сервисами,None,от 225 000 ₽,Старший инженер Linux,"Администратор серверов, Старший (Senior)","[Linux, Puppet, Ansible, CI/CD]",225000,225000,225000.0,₽,225000.0,4,56250.000000,Senior
2,Диванчик,None,от 200 000 до 230 000 ₽,Программист-аналитик 1C:ERP,Программист 1С,"[Разработка под 1С, 1C: ERP]",200000,230000,215000.0,₽,215000.0,2,107500.000000,NaN
3,Wanted.,​4.71​,до 200 000 ₽,С# Backend developer,Бэкенд разработчик,"[C#, .NET, PostgreSQL, Apache Kafka, Greenplum...",200000,200000,200000.0,₽,200000.0,6,33333.333333,NaN
4,Wanted.,​4.71​,до 120 000 ₽,Графический дизайнер упаковки,"Графический дизайнер, Средний (Middle)","[Дизайн упаковки, Допечатная подготовка]",120000,120000,120000.0,₽,120000.0,2,60000.000000,Middle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,MST,None,от 500 $,Саппорт в команду,"Менеджер технической поддержки, Младший (Junior)","[DNS, Техническая поддержка, Администрирование...",500,500,500.0,$,50000.0,4,125.000000,Junior
565,DevCube Innovations,None,от 2600 до 3000 $,.NET Developer – Remote - POS or Hotel Managem...,"Фулстек разработчик, Старший (Senior)","[C#, .NET, SQL, Английский язык, XML]",2600,3000,2800.0,$,280000.0,5,560.000000,Senior
566,Uptrade,None,от 130 000 до 180 000 ₽,.NET разработчик,"Бэкенд разработчик, Средний (Middle)","[Git, .NET Core, C#, ООП, PostgreSQL, .NET]",130000,180000,155000.0,₽,155000.0,6,25833.333333,Middle
567,STRONGTEAM,None,от 50 000 ₽,Junior Java Developer / Младший Java-разработчик,"Бэкенд разработчик, Младший (Junior)","[Java, Java Persistence API, ООП, Spring Boot,...",50000,50000,50000.0,₽,50000.0,10,5000.000000,Junior


In [5]:
# Создаем матрицу скиилов. На пересечении i-ой строки (вакансии) и j-ого столбца (скилла)
# значение равно 1,если текущий скилл представлен в данной вакансии и 0, если нет

skills_array = df.explode('Skills')['Skills'].unique()

skills_matrix = np.zeros((len(df),len(skills_array)))
skills_matrix.shape

(569, 499)

In [6]:
salary_array = df['RubAvgSalary']
len(salary_array)

569

In [7]:
for i, row in df.iterrows():
    for j, skill in enumerate(skills_array):
        if any(value == skill for value in row['Skills']):
            skills_matrix[i,j] = 1

In [8]:
# Решаем уравнение вида A*X = B. Где A - skills_matrix, B - массив зарплат.
# Решением является массив X - ожидаемая оценка зарплаты текущей зарплаты

x = np.linalg.lstsq(skills_matrix, np.array(salary_array), rcond=None)[0]

In [13]:
df_salary = pd.DataFrame(columns=['Skill','ExpSalary'])

for i, value in enumerate(x):
    df_salary.loc[len(df_salary)] = [skills_array[i],value]

In [14]:
# Выводим пары (навык; ожидаемая оценка)
# На текущий момент оценка происходит приоритетным способом.
# Больше приоритет - более высокая корреляция с высокой зарплатой
df_salary

,Skill,ExpSalary
0,Linux,150908.796655
1,Ansible,199881.124061
2,Puppet,-178128.424742
3,CI/CD,59677.008053
4,Разработка под 1С,163977.803148
...,...,...
494,Разработка решений по интеграции,9756.631787
495,ASP.NET MVC,175139.478025
496,Администрирование сайтов,-91664.664899
497,Поддержка сайтов,-91664.664899


In [15]:
# Можно сохранить
df_salary.to_csv('SkillExpSalary.csv')